In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from utils import *
import json
import random
#from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish

from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import classification_report, accuracy_score, make_scorer


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline


In [10]:
#column = ['tweets','duygu','preprocessing']
#df = pd.read_excel("../dataset/total.xlsx")

column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")


df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [11]:
df.loc[df.duygu=="olumlu","duygu"]=1
df.loc[df.duygu=="olumsuz","duygu"]=0
df[:4]

,tweets,duygu
0,turkcell heryerde çekiyor kesin bilgi,1
1,turkcell olmak ayrıcalıktir çünkü kuzenlerin v...,1
2,allahtan turkcell'liyim amin,1
3,avea kaşar yaşasın turkcell,1


In [12]:
df=df.sample(frac=1).reset_index(drop=True)
df

,tweets,duygu
0,artık dm lere bakmıyorum o bile turkcell den g...,0
1,az önce benim eskisi mesaj attı yok geri dön f...,1
2,@vaampirellaa ebeni sikem turkcell sdfdfds,0
3,bir tek benim mı internetim bozuk turkcell ben...,0
4,ulan dalga geciyorum kiziyorum felan ama tek m...,1
...,...,...
9121,rt @bursasporfann: bursa'da birçok turkcell ba...,1
9122,turkcell superonline pişmanlıktır hiç bir şeki...,0
9123,@eyganjaa turkcell sağolsun 18gb,1
9124,turkcell ne yaptın ne ettin beni kaybettin bil...,0


In [13]:
X_t=df['tweets'].tolist()

In [14]:
# Creating a tokenizer
tokenizer = Tokenizer(lower=True)
# Building word indices
tokenizer.fit_on_texts(X_t)
# Tokenizing sentences
sentences = tokenizer.texts_to_sequences(X_t)
# Creating a reverse dictionary
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)
# Creating texts 
X_t = list(map(sequence_to_text, sentences))


In [15]:
from gensim.models import Word2Vec
import time
start=time.time()
n_features=32
window_size=5
min_count=1
epoch=50
n_workers=8

wordembeddings = Word2Vec(size = n_features,
            window = window_size, 
            min_count= min_count,
            workers = n_workers, 
            sg=1)
wordembeddings.build_vocab(X_t)
wordembeddings.train(X_t, 
            total_examples=wordembeddings.corpus_count,  
            epochs = epoch)
end=time.time()
print(end-start)

32.60600161552429


In [16]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [17]:
def word2vec (X_train,y_train,X_test,y_test):
    #Create a tokenizer, configured to only take into account the 20 most common words çok küçük olursa kelimeleri 
    #kaybederiz underfit yaparız
    #tokenizer = Tokenizer(lower=True) #en yaygın kaç kelimeyi dikkate alacağı. Belirtilecek en iyi kelime sayısı #1000 yapan da var
    tokenizer.fit_on_texts(X_train) #keras tokenizer ile metni dictionary haline getiriyor.
    sequences_X_train = tokenizer.texts_to_sequences(X_train) #kelimelerin dictionarydeki karşılığı 
    #[[2, 1, 3], [2, 1], [4, 1], [5, 6]] şekline getiriliyor. 2-machine 1- learning 3-Knowledge 
    word_index = tokenizer.word_index #dictionarydeki kelimelerin sayısal karşılığı 'unk': 1, 'ürün': 2,
    max_length = 0
    for review_number in range(len(sequences_X_train)): #len(sequences_X_train) ile kaç tane [[2,3,4],[2,6]] var bulunuyor burda 2
        numberofwords=len(sequences_X_train[review_number]) #[2,3,4] içinde kaç tane şey var 3 burda
        if (numberofwords) > (max_length):
            max_length = numberofwords #tüm kelimelere bakıp en uzun olanı buluyor

    X_train = pad_sequences(sequences_X_train, maxlen=max_length) #ikili boyutlu matrise çevirip her cümelnin uzunluğunu eşit yapıyor.
    #En uzun cümle uzunluğuna tamamlanıyor.[[2 1 3] [0 2 1]] alt alta gelecek şekilde en uzun 6 ise 6x6 matris oluyor
    y_train = np.asarray(y_train) #tek boyutlu bir matris oluyor [1 1 0 ... 0 1 0] gibi

    sequences_X_test = tokenizer.texts_to_sequences(X_test) #train için yapılan gibi dictionary alınıyor
    X_test = pad_sequences(sequences_X_test, maxlen=max_length) #en uzun olana göre pad sequence yapılıyor
    y_test = np.asarray(y_test)


    unique_words = len(word_index) #word_index ile unique olan kelimeler alınıyor 0 dan başladığı için bir arttırılıyor
    total_words = unique_words + 1
    skipped_words = 0
    embedding_dim = 32 #embedding dim vector size ile aynı 
    embedding_vector=0
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items(): #kelime ve kelimenin dictionarydeki karşılığı alınıyor
        try:
            embedding_vector = wordembeddings[word]#wordembeddings.word_vectors[wordembeddings.dictionary[word]] #kelimenin word2vec karşılığı vektör olarak

        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector #dictionarydeki indexine word2vec teki sayısal hali yazılır
            
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)
    
    return embedding_layer,X_train,y_train,X_test,y_test

In [18]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]
pat = 5
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= word2vec (X_train,y_train,X_test,y_test)
    
    modelCNNLSTM = Sequential() #cnn

    modelCNNLSTM.add(embedding_layer)
    modelCNNLSTM.add(Conv1D(filters=256, kernel_size=5, activation='relu'))
    modelCNNLSTM.add(MaxPooling1D(pool_size=4))
    modelCNNLSTM.add(Dropout(0.25))
    modelCNNLSTM.add(LSTM(128))                            
    modelCNNLSTM.add(Flatten())
    modelCNNLSTM.add(Dense(1, activation='sigmoid'))
    modelCNNLSTM.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNNLSTM.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelCNNLSTM.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

<ipython-input-17-7d0886ed6bc1>:32: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  embedding_vector = wordembeddings[word]#wordembeddings.word_vectors[wordembeddings.dictionary[word]] #kelimenin word2vec karşılığı vektör olarak


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 6s 24ms/step - loss: 0.5278 - acc: 0.7314 - precision_1: 0.7379 - recall_1: 0.7204
Epoch 2/50
257/257 [==============================] - 6s 24ms/step - loss: 0.4558 - acc: 0.7824 - precision_1: 0.7970 - recall_1: 0.7597
Epoch 3/50
257/257 [==============================] - 6s 23ms/step - loss: 0.4131 - acc: 0.8058 - precision_1: 0.8170 - recall_1: 0.7898
Epoch 4/50
257/257 [==============================] - 6s 23ms/step - loss: 0.3589 - acc: 0.8327 - precision_1: 0.8395 - recall_1: 0.8240 0s - loss: 0.3565 - acc: 0.8333 - precision_1: 0.8401 
Epoch 5/50
257/257 [==============================] - 9s 37ms/step - loss: 0.3058 - acc: 0.8674 - precision_1: 0.8732 - recall_1: 0.8607
Epoch 6/50
257/257 [==============================] - 10s 37ms/step - loss: 0.2447 - acc: 0.8924 - precision_1: 0.8973 - recall_1: 0.8869
Epoch 7/50
257/257 [=====

257/257 [==============================] - 8s 31ms/step - loss: 0.2098 - acc: 0.9078 - precision_2: 0.9052 - recall_2: 0.9116
Epoch 8/50
257/257 [==============================] - 7s 26ms/step - loss: 0.1736 - acc: 0.9276 - precision_2: 0.9274 - recall_2: 0.9281
Epoch 9/50
257/257 [==============================] - 7s 26ms/step - loss: 0.1500 - acc: 0.9380 - precision_2: 0.9362 - recall_2: 0.9405
Epoch 10/50
257/257 [==============================] - 6s 25ms/step - loss: 0.1316 - acc: 0.9474 - precision_2: 0.9475 - recall_2: 0.9475
Epoch 11/50
257/257 [==============================] - 7s 29ms/step - loss: 0.1079 - acc: 0.9579 - precision_2: 0.9586 - recall_2: 0.9573
Epoch 12/50
257/257 [==============================] - 8s 30ms/step - loss: 0.1030 - acc: 0.9579 - precision_2: 0.9580 - recall_2: 0.9580 0s - loss: 0.1035 - acc: 0.9577 - precision_2: 0.9582 - recall_2: 
Epoch 13/50
257/257 [==============================] - 7s 26ms/step - loss: 0.0907 - acc: 0.9659 - precision_2: 0.9674 

257/257 [==============================] - 7s 26ms/step - loss: 0.5307 - acc: 0.7365 - precision_4: 0.7366 - recall_4: 0.7391
Epoch 2/50
257/257 [==============================] - 7s 28ms/step - loss: 0.4650 - acc: 0.7762 - precision_4: 0.7857 - recall_4: 0.7617
Epoch 3/50
257/257 [==============================] - 9s 33ms/step - loss: 0.4176 - acc: 0.8024 - precision_4: 0.8114 - recall_4: 0.7896
Epoch 4/50
257/257 [==============================] - 8s 33ms/step - loss: 0.3782 - acc: 0.8243 - precision_4: 0.8306 - recall_4: 0.8163
Epoch 5/50
257/257 [==============================] - 9s 35ms/step - loss: 0.3300 - acc: 0.8456 - precision_4: 0.8500 - recall_4: 0.8405
Epoch 6/50
257/257 [==============================] - 9s 34ms/step - loss: 0.2661 - acc: 0.8866 - precision_4: 0.8899 - recall_4: 0.8833
Epoch 7/50
257/257 [==============================] - 9s 33ms/step - loss: 0.2242 - acc: 0.9039 - precision_4: 0.9067 - recall_4: 0.9012
Epoch 8/50
257/257 [==============================] 

257/257 [==============================] - 11s 42ms/step - loss: 0.0887 - acc: 0.9627 - precision_6: 0.9642 - recall_6: 0.9609
Epoch 15/50
257/257 [==============================] - 10s 41ms/step - loss: 0.0786 - acc: 0.9686 - precision_6: 0.9719 - recall_6: 0.9648
Epoch 16/50
257/257 [==============================] - 11s 41ms/step - loss: 0.0735 - acc: 0.9733 - precision_6: 0.9750 - recall_6: 0.97145s - loss: 0.071
Epoch 17/50
257/257 [==============================] - 11s 42ms/step - loss: 0.0644 - acc: 0.9763 - precision_6: 0.9770 - recall_6: 0.9753
Epoch 18/50
257/257 [==============================] - 11s 41ms/step - loss: 0.0660 - acc: 0.9736 - precision_6: 0.9731 - recall_6: 0.9738
Epoch 19/50
257/257 [==============================] - 11s 41ms/step - loss: 0.0690 - acc: 0.9735 - precision_6: 0.9747 - recall_6: 0.97190s - loss: 0.0660 - acc: 0.9746 - precision_6: 0.9757 - 
Epoch 20/50
257/257 [==============================] - 10s 40ms/step - loss: 0.0498 - acc: 0.9820 - precis

257/257 [==============================] - 11s 41ms/step - loss: 0.0413 - acc: 0.9866 - precision_7: 0.9880 - recall_7: 0.9851
Epoch 32/50
257/257 [==============================] - 11s 42ms/step - loss: 0.0358 - acc: 0.9865 - precision_7: 0.9882 - recall_7: 0.9846
Epoch 33/50
257/257 [==============================] - 10s 41ms/step - loss: 0.0435 - acc: 0.9825 - precision_7: 0.9848 - recall_7: 0.9800
Epoch 34/50
257/257 [==============================] - 11s 41ms/step - loss: 0.0421 - acc: 0.9838 - precision_7: 0.9848 - recall_7: 0.9827
Epoch 35/50
257/257 [==============================] - 11s 43ms/step - loss: 0.0398 - acc: 0.9853 - precision_7: 0.9877 - recall_7: 0.9827
Epoch 36/50
257/257 [==============================] - 10s 41ms/step - loss: 0.0364 - acc: 0.9869 - precision_7: 0.9880 - recall_7: 0.9856
Epoch 37/50
257/257 [==============================] - 11s 41ms/step - loss: 0.0340 - acc: 0.9867 - precision_7: 0.9892 - recall_7: 0.9841
Epoch 38/50
257/257 [==================

257/257 [==============================] - 11s 43ms/step - loss: 0.0378 - acc: 0.9859 - precision_8: 0.9898 - recall_8: 0.9821
Epoch 37/50
257/257 [==============================] - 11s 42ms/step - loss: 0.0323 - acc: 0.9869 - precision_8: 0.9881 - recall_8: 0.9857
Epoch 38/50
257/257 [==============================] - 11s 42ms/step - loss: 0.0355 - acc: 0.9867 - precision_8: 0.9886 - recall_8: 0.9850
Epoch 39/50
257/257 [==============================] - 11s 43ms/step - loss: 0.0418 - acc: 0.9843 - precision_8: 0.9869 - recall_8: 0.98192s - l
Epoch 40/50
257/257 [==============================] - 11s 41ms/step - loss: 0.0335 - acc: 0.9883 - precision_8: 0.9915 - recall_8: 0.9853
Epoch 41/50
257/257 [==============================] - 11s 42ms/step - loss: 0.0375 - acc: 0.9865 - precision_8: 0.9881 - recall_8: 0.9850
Epoch 42/50
257/257 [==============================] - 11s 43ms/step - loss: 0.0359 - acc: 0.9860 - precision_8: 0.9898 - recall_8: 0.9824
Epoch 00042: early stopping
29/29

257/257 [==============================] - 11s 43ms/step - loss: 0.1511 - acc: 0.9374 - precision_10: 0.9384 - recall_10: 0.9365
Epoch 10/50
257/257 [==============================] - 11s 44ms/step - loss: 0.1257 - acc: 0.9489 - precision_10: 0.9509 - recall_10: 0.94676s - loss: 0.1121 - acc: 0.9566 - precision_10: 0.9614 - ETA: 5s - l - ETA: 2s - l
Epoch 11/50
257/257 [==============================] - 12s 45ms/step - loss: 0.1075 - acc: 0.9567 - precision_10: 0.9583 - recall_10: 0.95505s - l
Epoch 12/50
257/257 [==============================] - 11s 43ms/step - loss: 0.0974 - acc: 0.9621 - precision_10: 0.9641 - recall_10: 0.9601
Epoch 13/50
257/257 [==============================] - 11s 44ms/step - loss: 0.0863 - acc: 0.9663 - precision_10: 0.9678 - recall_10: 0.9647
Epoch 14/50
257/257 [==============================] - 11s 45ms/step - loss: 0.0836 - acc: 0.9654 - precision_10: 0.9684 - recall_10: 0.9623
Epoch 15/50
257/257 [==============================] - 12s 45ms/step - loss: 0

In [19]:
def Average(lst):
    return sum(lst) / len(lst)

In [20]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.764959
test precision: 0.776274
test recall: 0.747418
test f1_score: 0.760510
